In [ ]:
# 예제 2: 데이터 분석 어시스턴트 - 한국 주식 분석 Agent
# Google Colab에서 실행하세요

# ============================
# 1. 필요한 패키지 설치
# ============================
!pip -q install langchain langchain-openai langchain-community FinanceDataReader gradio matplotlib seaborn duckduckgo-search

In [17]:
# ============================
# 2. 라이브러리 임포트 및 설정
# ============================
import FinanceDataReader as fdr
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr
import pandas as pd
from datetime import datetime, timedelta
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage
from langchain_community.tools import DuckDuckGoSearchRun

import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

def set_font(): 
    # 폰트 설정
    font_path='/System/Library/Fonts/AppleGothic.ttf'
    fontprop = fm.FontProperties(fname=font_path, size=10)
    plt.rcParams['font.family'] = 'AppleGothic'  # macOS
    plt.rcParams['axes.unicode_minus'] = False
    
set_font()
# 한글 폰트 설정 (Colab용)
# try:
#     plt.rcParams['font.family'] = 'DejaVu Sans'
# except:
#     pass
# plt.rcParams['axes.unicode_minus'] = False

In [18]:
# ============================
# 3. API 키 설정 (Colab Secrets 또는 직접 입력)
# ============================
import os

# OpenAI API 키 설정 (사용자가 입력해야 함)
# from google.colab import userdata
# api_key=userdata.get('api_key')
# os.environ["OPENAI_API_KEY"] = api_key
# api_key2=userdata.get('api_key2')
# os.environ["LANGCHAIN_API_KEY"] = api_key2

from dotenv import load_dotenv

load_dotenv()
# OpenAI API 클라이언트 생성
OPENAPI_KEY = os.getenv("OPENAI_API_KEY")
LangSmith_KEY = os.getenv("LANGCHAIN_API_KEY")

# 2) LangSmith 연동 필수 환경변수
os.environ["LANGCHAIN_TRACING_V2"] = "true"      # 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"]   = "https://api.smith.langchain.com"  # 기본값
os.environ["LANGSMITH_PROJECT"]    = "Korean_Stock_Agent"                 # 수업용 프로젝트명

In [19]:
# ============================================
# 4. 한국 주식 데이터 수집 Tool 정의 (FinanceDataReader 사용)
# ============================================
def get_korean_stock_data(stock_code: str, period_days: int = 30) -> str:
    """
    한국 주식 데이터 (FinanceDataReader 사용)
    stock_code 예시: '005930' (삼성전자), '000660' (SK하이닉스), '035420' (NAVER), '051910' (LG화학)
    period_days: 최근 몇 일간의 데이터를 가져올지 (기본 30일)
    """
    try:
        # 종료일은 오늘, 시작일은 period_days 전
        end_date = datetime.now()
        start_date = end_date - timedelta(days=period_days)
        
        # FinanceDataReader로 한국 주식 데이터 가져오기
        df = fdr.DataReader(stock_code, start_date, end_date)
        
        if df.empty:
            return f"종목코드 {stock_code}의 데이터를 가져올 수 없습니다."
        
        # 최신 데이터 가져오기
        latest = df.tail(1)
        close_price = latest["Close"].values[0]
        
        # 주식 정보 가져오기 (종목명)
        try:
            stock_info = fdr.StockListing('KRX')
            stock_name = stock_info[stock_info['Code'] == stock_code]['Name'].values
            name = stock_name[0] if len(stock_name) > 0 else stock_code
        except:
            name = stock_code
        
        # 간단한 통계 정보
        high_price = df['High'].max()
        low_price = df['Low'].min()
        avg_price = df['Close'].mean()
        
        return f"""종목: {name} ({stock_code})
최근 종가: {close_price:,.0f}원
{period_days}일간 최고가: {high_price:,.0f}원
{period_days}일간 최저가: {low_price:,.0f}원
{period_days}일간 평균가: {avg_price:,.0f}원"""
        
    except Exception as e:
        return f"에러 발생: {e}"

# 한국 주요 종목 정보 조회 함수 (매개변수 수정)
def get_major_stocks_info(query: str = "") -> str:
    """
    한국 주요 종목 정보를 제공합니다.
    query: 사용되지 않지만 Agent 호환성을 위해 추가
    """
    major_stocks = {
        '005930': '삼성전자',
        '000660': 'SK하이닉스', 
        '035420': 'NAVER',
        '051910': 'LG화학',
        '006400': '삼성SDI',
        '035900': 'JYP Ent',
        '028260': '삼성물산',
        '068270': '셀트리온',
        '207940': '삼성바이오로직스',
        '323410': '카카오뱅크'
    }
    
    result = "한국 주요 종목 코드:\n"
    for code, name in major_stocks.items():
        result += f"- {code}: {name}\n"
    
    return result

# ============================================
# 5. DuckDuckGo를 이용한 뉴스 검색 기능
# ============================================

# DuckDuckGo 검색 도구 초기화
search = DuckDuckGoSearchRun()

def get_stock_news_ddg(stock_name: str, days: int = 7) -> str:
    """
    DuckDuckGo를 사용하여 주식 관련 뉴스를 검색합니다.
    stock_name: 회사명 (예: '삼성전자', 'SK하이닉스', 'NAVER' 등)
    days: 최근 몇 일간의 뉴스 (기본 7일)
    """
    try:
        # 검색 쿼리 생성 (한글과 영어 모두 검색)
        search_queries = [
            f"{stock_name} 주식 뉴스",
            f"{stock_name} 주가",
            f"{stock_name} news stock"
        ]
        
        all_results = []
        
        for query in search_queries:
            try:
                result = search.run(query)
                if result and result.strip():
                    all_results.append(f"🔍 검색어: '{query}'\n{result}\n{'='*50}\n")
            except Exception as e:
                continue
        
        if not all_results:
            return f"{stock_name} 관련 뉴스를 찾을 수 없습니다."
        
        # 결과 통합 및 요약
        combined_result = f"{stock_name} 관련 최신 뉴스 검색 결과:\n\n"
        combined_result += "\n".join(all_results[:2])  # 처음 2개 검색 결과만 표시
        
        return combined_result
        
    except Exception as e:
        return f"뉴스 검색 중 오류 발생: {e}"

# 종목코드로 회사명 조회 함수
def get_company_name_by_code(stock_code: str) -> str:
    """
    종목코드로 회사명을 조회합니다.
    """
    try:
        stock_info = fdr.StockListing('KRX')
        stock_name = stock_info[stock_info['Code'] == stock_code]['Name'].values
        return stock_name[0] if len(stock_name) > 0 else stock_code
    except:
        # 주요 종목 매핑
        major_stocks = {
            '005930': '삼성전자',
            '000660': 'SK하이닉스', 
            '035420': 'NAVER',
            '051910': 'LG화학',
            '006400': '삼성SDI',
            '035900': 'JYP',
            '028260': '삼성물산',
            '068270': '셀트리온',
            '207940': '삼성바이오로직스',
            '323410': '카카오뱅크'
        }
        return major_stocks.get(stock_code, stock_code)

def get_stock_news_by_code_ddg(stock_code: str, days: int = 7) -> str:
    """
    종목코드로 DuckDuckGo 뉴스를 검색합니다.
    """
    company_name = get_company_name_by_code(stock_code)
    return get_stock_news_ddg(company_name, days)

# Tool 정의
korean_stock_tool = Tool(
    name="get_korean_stock_data",
    description="한국 주식 종목코드와 기간(일수)을 입력받아 최근 주가 정보를 요약합니다. 예: '005930' (삼성전자)",
    func=get_korean_stock_data
)

major_stocks_tool = Tool(
    name="get_major_stocks_info",
    description="한국 주요 종목들의 코드와 이름을 제공합니다.",
    func=get_major_stocks_info
)

stock_news_ddg_tool = Tool(
    name="get_stock_news_ddg",
    description="DuckDuckGo를 사용해 회사명으로 최신 주식 뉴스를 검색합니다. 예: '삼성전자'",
    func=get_stock_news_ddg
)

stock_news_by_code_ddg_tool = Tool(
    name="get_stock_news_by_code_ddg",
    description="DuckDuckGo를 사용해 종목코드로 최신 주식 뉴스를 검색합니다. 예: '005930'",
    func=get_stock_news_by_code_ddg
)

# ============================================
# 6. Agent 설정
# ============================================
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

tools = [korean_stock_tool, major_stocks_tool, stock_news_ddg_tool, stock_news_by_code_ddg_tool]

prompt = ChatPromptTemplate.from_messages([
    ("system", """너는 한국 주식 분석 어시스턴트이다.
사용자가 요청하면 FinanceDataReader를 사용해 한국 주식 데이터를 가져오고 간단한 분석을 제공한다.
또한 DuckDuckGo 검색을 통해 관련 종목의 최신 뉴스를 검색해서 제공할 수 있다.

주요 기능:
1. 종목코드는 6자리 숫자로 입력받는다 (예: 005930은 삼성전자)
2. 사용자가 종목명을 물어보면 주요 종목 정보를 먼저 제공한다
3. 구체적인 종목코드를 입력하면 해당 주식 데이터를 분석한다
4. 뉴스 검색 요청시 DuckDuckGo를 통해 해당 종목의 최신 뉴스를 제공한다
5. 종목코드로 뉴스를 검색할 수도 있다

사용자가 뉴스를 요청하면 DuckDuckGo 검색 결과를 바탕으로 최신 정보를 제공한다."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# ============================================
# 7. Gradio용 함수 정의
# ============================================
def process_query(message, history):
    chat_history = []
    if history:
        for h in history:
            if h[0]: chat_history.append(HumanMessage(content=h[0]))
            if h[1]: chat_history.append(AIMessage(content=h[1]))

    response = agent_executor.invoke({
        "input": message,
        "chat_history": chat_history
    })
    return response["output"]

def plot_korean_stock(stock_code: str, period_days: int = 30):
    try:
        # 날짜 설정
        end_date = datetime.now()
        start_date = end_date - timedelta(days=period_days)
        
        # FinanceDataReader로 데이터 가져오기
        df = fdr.DataReader(stock_code, start_date, end_date)
        
        if df.empty:
            return None, f"{stock_code} 데이터 없음"
        
        # 종목명 가져오기
        try:
            stock_info = fdr.StockListing('KRX')
            stock_name = stock_info[stock_info['Code'] == stock_code]['Name'].values
            name = stock_name[0] if len(stock_name) > 0 else stock_code
        except:
            name = stock_code
        
        # 그래프 생성
        plt.figure(figsize=(12, 6))
        
        # 상단: 주가 차트 (종가, 고가, 저가)
        plt.subplot(1, 1, 1)
        plt.plot(df.index, df['Close'], label='종가', linewidth=2, color='blue')
        plt.fill_between(df.index, df['Low'], df['High'], alpha=0.3, color='lightblue', label='고가-저가 범위')
        plt.title(f'{name} ({stock_code}) - {period_days}일간 주가 추이', fontsize=16, fontweight='bold')
        plt.ylabel('주가 (원)', fontsize=12)
        plt.legend()
        plt.grid(True, alpha=0.3)

        
        plt.tight_layout()
        
        # 이미지 저장
        img_path = f"{stock_code}_{period_days}days.png"
        plt.savefig(img_path, dpi=300, bbox_inches='tight')
        plt.close()
        
        # 분석 결과 생성
        latest_price = df['Close'].iloc[-1]
        high_price = df['High'].max()
        low_price = df['Low'].min()
        avg_volume = df['Volume'].mean()
        
        analysis = f"""{name} ({stock_code}) {period_days}일간 분석:
• 최근 종가: {latest_price:,.0f}원
• 최고가: {high_price:,.0f}원
• 최저가: {low_price:,.0f}원  
• 평균 거래량: {avg_volume:,.0f}주
• 변동률: {((latest_price - df['Close'].iloc[0]) / df['Close'].iloc[0] * 100):+.2f}%"""
        
        return img_path, analysis
        
    except Exception as e:
        return None, f"그래프 생성 실패: {e}"

# 뉴스 가져오기 함수 (UI용)
def get_news_for_ui(stock_code: str):
    if stock_code and stock_code.strip():
        return get_stock_news_by_code_ddg(stock_code, 7)
    else:
        return "종목코드를 입력하세요."

# ============================================
# 8. Gradio 인터페이스
# ============================================
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 📈 한국 주식 분석 챗봇 (DuckDuckGo 뉴스 검색)\nLangChain + Agent + Gradio + FinanceDataReader + DuckDuckGo")
    
    gr.Markdown("""
    ### 사용법:
    - **채팅**: "삼성전자 주가 알려줘", "삼성전자 뉴스 알려줘", "주요 종목 코드 알려줘"
    - **그래프**: 종목코드 입력 후 그래프 생성 (예: 005930은 삼성전자)
    - **뉴스**: 종목코드 입력 후 DuckDuckGo에서 최신 뉴스 검색
    - **주요 종목**: 005930(삼성전자), 000660(SK하이닉스), 035420(NAVER) 등
    
    ⚠️ **참고**: 뉴스는 DuckDuckGo 검색 엔진을 통해 실시간으로 검색됩니다.
    """)

    # 채팅 인터페이스
    chatbot = gr.Chatbot(height=300)
    msg = gr.Textbox(label="질문 입력", placeholder="예: 삼성전자 주가와 뉴스 알려줘")
    
    with gr.Row():
        stock_code_box = gr.Textbox(label="종목코드 (6자리)", value="005930", placeholder="예: 005930")
        period_box = gr.Dropdown(
            ["7", "30", "90", "180", "365"], value="30", label="기간 (일)"
        )
        
    # 결과 표시 영역
    with gr.Row():
        with gr.Column():
            graph = gr.Image(label="주가 그래프")
            info = gr.Textbox(label="분석 결과")
        with gr.Column():
            news_display = gr.Textbox(label="최신 뉴스 (DuckDuckGo)", lines=10)
            news_btn = gr.Button("뉴스 새로고침")

    def respond(message, chat_history, stock_code, period_days):
        # 에이전트 응답 처리
        agent_answer = process_query(message, chat_history)
        
        # 그래프 생성 (종목코드가 입력된 경우)
        img, desc = None, "그래프를 생성하려면 종목코드를 입력하세요."
        if stock_code and stock_code.strip():
            img, desc = plot_korean_stock(stock_code, int(period_days))
        
        # 뉴스 가져오기
        news = get_news_for_ui(stock_code) if stock_code and stock_code.strip() else "뉴스를 보려면 종목코드를 입력하세요."
        
        chat_history.append((message, agent_answer))
        return "", chat_history, img, desc, news

    # 뉴스만 새로고침하는 함수
    def refresh_news(stock_code):
        return get_news_for_ui(stock_code)

    # 이벤트 연결
    msg.submit(respond, [msg, chatbot, stock_code_box, period_box], [msg, chatbot, graph, info, news_display])
    news_btn.click(refresh_news, [stock_code_box], [news_display])

# ============================================
# 9. 실행
# ============================================
if __name__ == "__main__":
    demo.launch(share=True, debug=True)

/var/folders/xn/w4kqb1715fz7h49tp_h7hbmr0000gn/T/ipykernel_69417/3242317474.py:298: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=300)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://22f3feef25fa49f47b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...

Invoking: `get_korean_stock_data` with `005930`


종목: 삼성전자 (005930)
최근 종가: 69,100원
30일간 최고가: 72,400원
30일간 최저가: 67,500원
30일간 평균가: 70,343원
Invoking: `get_stock_news_ddg` with `삼성전자`




/Users/kwangsiklee/miniforge3/envs/langchain/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/Users/kwangsiklee/miniforge3/envs/langchain/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/Users/kwangsiklee/miniforge3/envs/langchain/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


삼성전자 관련 최신 뉴스 검색 결과:

🔍 검색어: '삼성전자 주식 뉴스'
1 day ago · 위기에 놓인 국내 반도체 기업 (사진=연합뉴스) 국내 반도체가 8월에 역대 최대의 월별 수출실적을 기록했다고 발표된 1일 중국과 미국발 악재로 삼성전자와 SK하이닉스 주가는 곤두박질쳤다. 이날 … 1 day ago · 삼성전자 로고 KB증권이 삼성전자에 대해 ‘매수’ 의견과 목표주가 9만 원을 유지했다. 지난달 28일 김동원 연구원은 “1c나노 기반 HBM4 개발 성능이 양호하며, 내년 1분기 평택 신규 증설을 통해 … Jul 28, 2025 · 삼성전자 (005930) 가 마침내 ‘7만전자’를 회복했다. 28일 한국거래소에 따르면 이날 오후 3시 8분 현재 삼성전자의 주가는 전 거래일 대비 4100원 (6.22%) 오른 7만 원에 거래 … Jul 28, 2025 · (서울=연합뉴스) 조민정 기자 = 삼성전자 [005930]가 28일 작년 9월 이후 약 11개월 만에 '7만전자'로의 복귀를 알렸다. 한국거래소에 따르면 삼성전자는 이날 전 거래일 대비 6.83% … 한화에어로스페이스의 최대 수혜자는 김승연 회장? 08.25 [외신 헤드라인] 美 정부, 인텔 최대주주 됐다…"지분 10% 소유" 08.25 "우리 반도체 기술, 中에 이미 따라잡혀…기술향상 시급" 08.25 삼성·LG, 'AI …

🔍 검색어: '삼성전자 주가'
6 days ago · [A005930] 삼성전자의 실시간 주가와 최신 뉴스, 분석 정보를 토스증권에서 한 번에 확인해보세요. Sep 20, 2024 · 투자업계에 따르면 최근 한 달간 삼성전자의 주가는 16.58% 떨어졌고, 지난 7월 11일 기록한 고점 8만 8,800원 대비 19일 기준 무려 28% 가까이 폭락한 6만 3,100원에 … Feb 26, 2025 · 삼성전자의 주가, 005930 주식, 차트, 기술적 분석, 실적 자료 등 69,700로 거래를 마감한 삼성전자 시세에 대한 자세한 정보를 확인해 보세요. 4 days ago · 현재 

...

/Users/kwangsiklee/miniforge3/envs/langchain/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/Users/kwangsiklee/miniforge3/envs/langchain/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/Users/kwangsiklee/miniforge3/envs/langchain/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


삼성전자 관련 최신 뉴스 검색 결과:

🔍 검색어: '삼성전자 주식 뉴스'
1 day ago · 위기에 놓인 국내 반도체 기업 (사진=연합뉴스) 국내 반도체가 8월에 역대 최대의 월별 수출실적을 기록했다고 발표된 1일 중국과 미국발 악재로 삼성전자와 SK하이닉스 주가는 곤두박질쳤다. 이날 … 1 day ago · 삼성전자 로고 KB증권이 삼성전자에 대해 ‘매수’ 의견과 목표주가 9만 원을 유지했다. 지난달 28일 김동원 연구원은 “1c나노 기반 HBM4 개발 성능이 양호하며, 내년 1분기 평택 신규 증설을 통해 … Jul 28, 2025 · 삼성전자 (005930) 가 마침내 ‘7만전자’를 회복했다. 28일 한국거래소에 따르면 이날 오후 3시 8분 현재 삼성전자의 주가는 전 거래일 대비 4100원 (6.22%) 오른 7만 원에 거래 … Jul 28, 2025 · (서울=연합뉴스) 조민정 기자 = 삼성전자 [005930]가 28일 작년 9월 이후 약 11개월 만에 '7만전자'로의 복귀를 알렸다. 한국거래소에 따르면 삼성전자는 이날 전 거래일 대비 … 한화에어로스페이스의 최대 수혜자는 김승연 회장? 08.25 [외신 헤드라인] 美 정부, 인텔 최대주주 됐다…"지분 10% 소유" 08.25 "우리 반도체 기술, 中에 이미 따라잡혀…기술향상 시급" 08.25 …

🔍 검색어: '삼성전자 주가'
[A005930] 삼성전자의 실시간 주가와 최신 뉴스, 분석 정보를 토스증권에서 한 번에 확인해보세요. Sep 20, 2024 · 투자업계에 따르면 최근 한 달간 삼성전자의 주가는 16.58% 떨어졌고, 지난 7월 11일 기록한 고점 8만 8,800원 대비 19일 기준 무려 28% 가까이 폭락한 6만 3,100원에 거래되고 있다. … 삼성전자의 주가, 005930 주식, 차트, 기술적 분석, 실적 자료 등 69,700로 거래를 마감한 삼성전자 시세에 대한 자세한 정보를 확인해 보세요. 4 days ago · 현재 주가는 56,600원입니다. 실시간 차트, 최신 뉴스, 분석 및 

/Users/kwangsiklee/miniforge3/envs/langchain/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/Users/kwangsiklee/miniforge3/envs/langchain/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/Users/kwangsiklee/miniforge3/envs/langchain/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://22f3feef25fa49f47b.gradio.live


/Users/kwangsiklee/miniforge3/envs/langchain/lib/python3.10/site-packages/gradio/tunneling.py:121: ResourceWarning: unclosed file <_io.BufferedReader name=83>
  self.proc = None
/Users/kwangsiklee/miniforge3/envs/langchain/lib/python3.10/site-packages/gradio/tunneling.py:121: ResourceWarning: unclosed file <_io.BufferedReader name=93>
  self.proc = None
